In [ ]:
from agno.agent import Agent
from agno.models.openai import OpenAILike
from agno.tools.duckduckgo import DuckDuckGoTools
from agno.workflow import Workflow
from dotenv import load_dotenv
from agno.workflow import Workflow, Step, StepInput
import os
from typing import Any
import json
from settings import OpenAISettings
from agno.tools.reasoning import ReasoningTools
from agno.db.sqlite import SqliteDb
from agno.vectordb.lancedb import LanceDb
from agno.knowledge.knowledge import Knowledge

load_dotenv()  




model = OpenAILike(
    id="openai/gpt-5-mini",
    supports_native_structured_outputs=True,
    base_url=os.environ["OPENAI_BASE_URL"],
    api_key=os.environ["OPENAI_API_KEY"],
    reasoning_effort="high",
)


# refine_agent = Agent(
#     model=model,
#     name="QueryRefiner",
#     tools=[],  # без внешних инструментов
#     instructions="Ты помощник, формулирующий короткие поисковые запросы. Верни одну строчку запроса.",
#     markdown=False,
#     stream=False,
#     reasoning=False,
# )

search_agent = Agent(
    model=model,
    name="DuckSearcher",
    tools=[DuckDuckGoTools()],
    instructions=(
        "Ты — агент, использующий DuckDuckGo. "
        "Всегда предоставляй итоговый ответ строго на русском языке, даже если источник на английском "
        "или если пользователь задал вопрос на русском/английском. "
        "Переводи всё найденное на русский язык перед выводом. "
        
    ),
    markdown=False,
    stream=False,
    reasoning=True,
    read_tool_call_history=True,
)
workflow = Workflow(
    name="duckduckgo_workflow",
    description="Поиск в интернете через DuckDuckGo",
    steps=[
        Step(
            name="search_step",
            agent=search_agent
        )
    ]
)

result = workflow.run("как зовут моргенштера настоящее имя??")

print(result.content)


WARNING  Failed to create Function for: duckduckgo_search

ValidationError: 1 validation error for Function
cache_results
  Input should be a valid boolean, unable to interpret input [type=bool_parsing, input_value=3, input_type=int]
    For further information visit https://errors.pydantic.dev/2.12/v/bool_parsing

In [ ]:
model = OpenAILike(
    id="openai/gpt-5-mini",
    supports_native_structured_outputs=True,
    base_url=os.environ["OPENAI_BASE_URL"],
    api_key=os.environ["OPENAI_API_KEY"],
    reasoning_effort="high",
)

class SimpleKnowledge:
    def __init__(self, filepath="tmp/knowledge.json"):
        self.filepath = filepath
        os.makedirs(os.path.dirname(filepath), exist_ok=True)
        self.data = self._load()
    
    def _load(self):
        if os.path.exists(self.filepath):
            with open(self.filepath, 'r', encoding='utf-8') as f:
                return json.load(f)
        return {}
    
    def _save(self):
        with open(self.filepath, 'w', encoding='utf-8') as f:
            json.dump(self.data, f, ensure_ascii=False, indent=2)
    
    def add(self, key, value):
        self.data[key] = value
        self._save()
    
    def get(self, key):
        return self.data.get(key)
    
    def search(self, query):
        query_lower = query.lower()
        results = []
        for key, value in self.data.items():
            if query_lower in key.lower() or query_lower in value.lower():
                results.append(f"{key}: {value}")
        return "\n".join(results) if results else "Информация не найдена в базе знаний."
    
    def get_all(self):
        return "\n".join([f"{k}: {v}" for k, v in self.data.items()])

knowledge = SimpleKnowledge()

print("Инициализация базы знаний...")
knowledge.add("Моргенштерн_имя", "Моргенштерн (настоящее имя Алишер Тагирович Валеев) — российский рэп-исполнитель, блогер и предприниматель.")
knowledge.add("Моргенштерн_дата_рождения", "Моргенштерн родился 17 февраля 1998 года.")
knowledge.add("Моргенштерн_возраст", "По состоянию на 2024 год Моргенштерну 26 лет.")
knowledge.add("Элджей_имя", "Элджей (настоящее имя Алексей Узенюк) — российский хип-хоп исполнитель, продюсер.")


storage = SqliteDb(
    db_file="tmp/agents.db"
)

search_agent = Agent(
    model=model,
    name="SmartSearcher",
    tools=[DuckDuckGoTools()],
    instructions=(
        "Ты — интеллектуальный поисковый агент с доступом к базе знаний и интернету.\n\n"
        f"БАЗА ЗНАНИЙ (проверяй СНАЧАЛА):\n{knowledge.get_all()}\n\n"
        "Твоя задача:\n"
        "1. СНАЧАЛА проверь базу знаний выше - если ответ там есть, используй его как приоритетный источник\n"
        "2. Если информации нет в базе или нужно уточнить/обновить - используй DuckDuckGo для поиска\n"
        "3. При поиске проверяй несколько источников для точности\n"
        "4. Синтезируй ответ на основе найденного\n"
        "5. ВАЖНО: Помни контекст предыдущих вопросов в рамках сессии\n\n"
        "ВАЖНО: \n"
        "- Всегда отвечай на русском языке, переводя информацию из источников\n"
        "- Давай КРАТКИЕ и СТРУКТУРИРОВАННЫЕ ответы (не более 300 слов)\n"
        "- Если находишь противоречивую информацию - укажи на это\n"
        "- Используй маркированные списки для лучшей читаемости\n"
        "- Используй информацию из предыдущих сообщений для контекста"
    ),
    markdown=True,
    stream=False,
    reasoning=True,
    read_tool_call_history=True,
    session_id="default_session",
)

workflow = Workflow(
    name="smart_search_workflow",
    description="Интеллектуальный поиск с планированием, памятью и базой знаний",
    steps=[
        Step(
            name="intelligent_search",
            agent=search_agent
        )
    ]
)

if __name__ == "__main__":
    session_id = "session"
    
    print("Запрос с использованием базы знаний")
    result1 = workflow.run(
        "как зовут моргенштера настоящее имя?",
        session_id=session_id
    )
    print(result1.content)
    
    print("\nproverochka na pamyat`")
    result2 = workflow.run(
        "А сколько ему лет?",
        session_id=session_id
    )
    print(result2.content)
   
    print("\nзапрос с reasoning")
    result3 = workflow.run(
        "Сравни карьеру Моргенштерна и Элджея: кто популярнее и почему?",
        session_id=session_id
    )
    
    content3 = result3.content
    print(content3)
    
   

In [ ]:
model = OpenAILike(
    id="openai/gpt-5-mini",
    supports_native_structured_outputs=True,
    base_url=os.environ["OPENAI_BASE_URL"],
    api_key=os.environ["OPENAI_API_KEY"],
    reasoning_effort="high",
)

class SimpleKnowledge:
    def __init__(self, filepath="tmp/knowledge.json"):
        self.filepath = filepath
        os.makedirs(os.path.dirname(filepath), exist_ok=True)
        self.data = self._load()
    
    def _load(self):
        if os.path.exists(self.filepath):
            with open(self.filepath, 'r', encoding='utf-8') as f:
                return json.load(f)
        return {}
    
    def _save(self):
        with open(self.filepath, 'w', encoding='utf-8') as f:
            json.dump(self.data, f, ensure_ascii=False, indent=2)
    
    def add(self, key, value):
        self.data[key] = value
        self._save()
    
    def get(self, key):
        return self.data.get(key)
    
    def search(self, query):
        query_lower = query.lower()
        results = []
        for key, value in self.data.items():
            if query_lower in key.lower() or query_lower in value.lower():
                results.append(f"{key}: {value}")
        return "\n".join(results) if results else "Информация не найдена в базе знаний."
    
    def get_all(self):
        return "\n".join([f"{k}: {v}" for k, v in self.data.items()])

knowledge = SimpleKnowledge()

knowledge.add("Моргенштерн_имя", "Моргенштерн (настоящее имя Алишер Тагирович Валеев) — российский рэп-исполнитель, блогер и предприниматель.")
knowledge.add("Моргенштерн_дата_рождения", "Моргенштерн родился 17 февраля 1998 года.")
knowledge.add("Моргенштерн_возраст", "По состоянию на 2024 год Моргенштерну 26 лет.")
knowledge.add("Элджей_имя", "Элджей (настоящее имя Алексей Узенюк) — российский хип-хоп исполнитель, продюсер.")

storage = SqliteDb(
    db_file="tmp/agents.db"
)


knowledge_agent = Agent(
    model=model,
    name="KnowledgeAgent",
    tools=[],
    instructions=(
        "Ты — агент базы знаний. Отвечаешь ТОЛЬКО на основе базы знаний.\n\n"
        f"БАЗА ЗНАНИЙ:\n{knowledge.get_all()}\n\n"
        "Правила:\n"
        "1. Если информация ЕСТЬ в базе - дай полный ответ\n"
        "2. Если информации НЕТ - скажи: 'ПОИСК_ТРЕБУЕТСЯ: [краткое описание что нужно найти]'\n"
        "3. Отслеживай контекст беседы для местоимений (он/она/его/её)\n"
        "4. Всегда на русском языке\n"
        "5. Будь кратким"
    ),
    markdown=False,
    stream=False,
    reasoning=True,
)


search_agent = Agent(
    model=model,
    name="SearchAgent",
    tools=[DuckDuckGoTools()],
    instructions=(
        "Ты — агент веб-поиска. Используй DuckDuckGo для поиска информации.\n\n"
        "Правила:\n"
        "1. Ищи только релевантную информацию\n"
        "2. Проверяй несколько источников\n"
        "3. Переводи всё на русский\n"
        "4. Будь кратким (до 200 слов)\n"
        "5. Отслеживай контекст для местоимений"
    ),
    markdown=False,
    stream=False,
    reasoning=True,
    read_tool_call_history=True,
)


coordinator_agent = Agent(
    model=model,
    name="Coordinator",
    tools=[],
    instructions=(
        "Ты — координатор мультиагентной системы.\n\n"
        "У тебя есть два помощника:\n"
        "1. KnowledgeAgent - работает с локальной базой знаний\n"
        "2. SearchAgent - ищет в интернете\n\n"
        "Твоя стратегия:\n"
        "1. Сначала спроси у KnowledgeAgent\n"
        "2. Если он ответил полностью - используй его ответ\n"
        "3. Если он сказал 'ПОИСК_ТРЕБУЕТСЯ' - спроси у SearchAgent\n"
        "4. Объедини результаты в финальный ответ\n"
        "5. Отслеживай контекст беседы (особенно местоимения)\n\n"
        "Приоритет: база знаний > веб-поиск\n"
        "Формат ответа: краткий, структурированный, на русском"
    ),
    markdown=True,
    stream=False,
    reasoning=True,
)


knowledge_workflow = Workflow(
    name="knowledge_only",
    description="Поиск только в базе знаний",
    steps=[
        Step(
            name="knowledge_search",
            agent=knowledge_agent
        )
    ]
)


full_search_workflow = Workflow(
    name="full_search",
    description="Поиск в базе знаний и интернете",
    steps=[
        Step(
            name="knowledge_search",
            agent=knowledge_agent
        ),
        Step(
            name="web_search",
            agent=search_agent
        )
    ]
)

def smart_search(query: str, session_id: str = "default"):
    
    result_knowledge = knowledge_workflow.run(query, session_id=session_id)
    
    if "ПОИСК_ТРЕБУЕТСЯ" in result_knowledge.content:
        print("  [Система] База знаний не содержит информации, запускаю веб-поиск...")
        result_full = full_search_workflow.run(query, session_id=session_id)
        return result_full.content
    else:
        print("  [Система] Информация найдена в базе знаний")
        return result_knowledge.content

if __name__ == "__main__":
    session_id = "multi_agent_session"
    
    
    print("Тест 1: Запрос из базы знаний")

    result1 = smart_search(
        "как зовут моргенштерна настоящее имя?",
        session_id=session_id
    )
    print(result1)

    print("Тест 2: Проверка памяти")

    result2 = smart_search(
        "А сколько ему лет?",
        session_id=session_id
    )
    print(result2)
    
   
    print("Тест 3: Запрос требующий гуглинга")

    result3 = smart_search(
        "Какие последние новости про Моргенштерна?",
        session_id=session_id
    )
    print(result3)
    print("Тест 3: Запрос требующий ризонинга")

    result3 = smart_search(
        "Сравни популярность Моргенштерна и Элджея?",
        session_id=session_id
    )
    print(result3)
    
    